1. Happy Number. Given a number, take square of all digits, and add them. If sum is 1, HAPPY, if it cycles, not.
2. Construct the largest number from an array.
3. Fast power. Comute $x^n$, _quickly_.
4. Longest increasing subsequence.
5. Skyline Problem.

---

# 1. Happy Number?

Can you determine, algorithmically, if it will be a happy number?

If the same pattern of digits repeats, discounting the number of zeros, then it will repeat endlessly ...

Can you prove that this will always terminate?
* For the largest 3 digit number `999`, the next number is `243`
* For the largest 4 digit number `9999`, the next number is `324`
* For the largest 10 digit number `9999....`, the next number if `810`

Eventually, the number will come down, regardless of where is starts, and it will either terminate, or cycle.

In [4]:
class HappyNumber:
    def solve(self, number):
        if number == 0:
            return False
        
        def sumsq(n):
            a = 0
            while n != 0:
                a += (n%10)**2
                n = n//10
            return a
        
        seen = set()
        def happy(n):
            x = sumsq(n)
            if x == 1:
                return True
            elif x in seen:
                return False
            else:
                seen.add(x)
                return happy(x)
            
        return happy(number)

In [8]:
o = HappyNumber()
o.solve(101)

False

# 2. Construct the largest number from an array.

Create a number by concatenating array elements, in any order.

One way is to consider all `n!` permutations.

But you know, a number starting from 9 is greater than a number starting from 1.

So, start by sorting numbers in reverse lex order -- everything beginning with `9*` is first, then `8*` and so on ...

In [30]:
class ConstructLargest:
    def solve(self, numbers):
        lex = list(sorted([str(n) for n in numbers], reverse=True))
        for i in range(len(lex)-1):
            first = lex[i]
            second = lex[i+1]
            if first + second < second + first:
                lex[i], lex[i+1] = second, first
                
        return int(''.join(lex))

In [31]:
o = ConstructLargest()
o.solve([9, 991, 8])

99918

In [20]:
'99' > '991'

False

In [21]:
'99991' > '99199'

True

In [19]:
'30' > '33'

False

# 3. Fast Power.

Compute $x^n$

In [32]:
class FastPower:
    def solve(self, x, n):
        assert n >= 1
        
        def power(e):
            if e == 1:
                return x
            else:
                half = power(e//2)
                if e%2 == 0:
                    return half * half
                else:
                    return half * half * x
                
        return power(n)

In [36]:
o = FastPower()
o.solve(2, 11)

2048

# 4. Longest Increasing Subsequence.

Given an array of randomly ordered number, find the length of the longest increasing subsequence.
```
[3, 4, 5, 1] => 3
[1, 6, 17, 2, 3, 15, 7] => 4
```

In [43]:
class LongestIncSub:
    def solve(self, nums):
        def sar(x, array):
            if len(array) == 1:
                array[0] = x
            else:
                start = 0
                end = len(array) - 1
                while start <= end:
                    mid = start + (end - start)//2
                    if array[mid] >= x:
                        end = mid - 1
                    else:
                        start = mid + 1
                        
                array[end+1] = x
        
        temp = []
        for n in nums:
            if temp:
                if n == temp[-1]:
                    pass
                elif n > temp[-1]:
                    temp.append(n)
                else:
                    # Search and replace.
                    sar(n, temp)
            else:
                temp.append(n)
                
        print(temp)
        return len(temp)

In [46]:
o = LongestIncSub()
print(o.solve([3, 4, 5, 1]))
print(o.solve([1, 6, 17, 2, 3, 15, 7]))

[1, 4, 5]
3
[1, 2, 3, 7]
4


# 5. Skyline Problem

Given building `(start, end, height)` triples, can you build a skyline? I.e. the (x, y) coordinate pairs where the height changes.

You are encountering buildings.

Every time you encounter a new building's start or end, you need to query the highest point at the moment.
And that will become the outline.

In [60]:
from collections import namedtuple

class Skyline:
    def solve(self, buildings):
        skyline = []
        tracker = []
        Building = namedtuple("building", ["start", "end", "height"])
        buildings = [Building(*b) for b in buildings]
        
        Point = namedtuple("point", ["position", "height", "building", "isStart"])
        data = sorted([Point(b.start, b.height, ix, True) for ix, b in enumerate(buildings)] +\
                      [Point(b.end, b.height, ix, False) for ix, b in enumerate(buildings)], 
                      key=lambda b:b.position)
        
        for p in data:
            if p.isStart:
                tracker.append(p)
                best = max([b.height for b in tracker])
            else:
                best = max([b.height for b in tracker if b.building != p.building] + [0])
                
            if skyline:
                if skyline[-1] != best:
                    skyline.append(best)
            else:
                skyline.append(best)
                
        return skyline

In [61]:
o = Skyline()
o.solve([(1, 3, 4), (2, 6, 8), (5, 10, 2), (12, 13, 1), (13, 14, 2)])

[4, 8, 4, 8]

In [63]:
1 + float('inf')

inf

6 qualifying courses + 1 elective

Any two SCS courses may count as qualifying.

1 course from each area.

1st sem:
* Advanced NLP (11-711): qualifying and Human Language area
* Intro to ML (10-701): qualifying and Machine Learning area
* Statistics (36-700): elective
* Elective done, 2/6 qualifying done, 1 SCS qualifying left

2nd Sem:
* PGM (10-708): qualifying, SCS
* Convex Optimization: extra
* Multimodal: qualifying
* 4/6 qualifying done, 0 SCS left, Applications area left

3rd Sem:
* Speech Processing 11-751: qualifying and Applications area
* Langauge And Statistics 11-611: qualifying

# 6. Jump Game III

https://leetcode.com/problems/jump-game-iii/

You are given an array of non-neg numbers and a starting location. From each position `i`, you can move `array[i]` steps to the left or to the right. Return true if you can eventually end in any position `j` where `array[j]` is 0.

In [71]:
from functools import lru_cache

class JumpGame3:
    def solve(self, arr, start):
        # sink = set([i for i, n in enumerate(arr) if n == 0])
        @lru_cache(maxsize=None)
        def search(i):
            if i >= len(arr) or i < 0:
                return False
            elif arr[i] == 0:
                return True
            else:
                val = arr[i]
                arr[i] = None
                for temp in [i+val, i-val]:
                    if temp >= 0 and temp < len(arr) and arr[temp] is not None and search(temp):
                        arr[i] = val
                        return True
                    
                arr[i] = val
                return False
            
        return search(start)

In [72]:
o = JumpGame3()
print(o.solve([4,2,3,0,3,1,2], 5))
print(o.solve([4,2,3,0,3,1,2], 0))

True
True


In [73]:
print(o.solve([3,0,2,1,2], 2))

False
